# Import

In [1]:
import wandb
import torch
import torch.nn as nn
import os
from tqdm import tqdm
from model.CNN import CNN
from util.checkpoints import save_checkpoint, load_checkpoint

device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

## random seed 고정 (무조건 1순위)
## 재현 가능하도록 하기 위해
## train dataset과 val dataset을 나눠놔야한다. 이건 seed를 고정해놔도 random split하면서 바뀜 
## 따라서, 동일한 환경에서 실험할 수 있도록 할 것

In [2]:
device

device(type='cuda')

# Settings

# DataLoader

In [3]:
import torch
from torch.utils.data import DataLoader, Dataset

In [4]:
BATCH_SIZE =  32 #한 배치당 32개 이미지데이터
EPOCHS = 40 # 전체 데이터 셋을 40번 반복
lr = 1e-2 # 학습률(learnign rate)

csv_file = ""
root_dir = "./data"

In [5]:
from torchvision import transforms
from util.data import CustomDataset

# 이미지 전처리 및 데이터셋 생성
transform = transforms.Compose([ ## Albumentation 사용하는 것으로 변경
    transforms.Resize((224, 224)),  # 원하는 크기로 리사이즈
    transforms.ToTensor(),           # 텐서로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 정규화
])

## shuffle은 false로 두고
## 데이터를 나눌 때 idx를 섞고 csv 파일을 나눠놓을 것 => train, val 비율 클래스별로 9:1
## k-fold => begging과 boosting을 찾아보면 좋을 것
## k-fold => 하나의 모델을 k번 학습하는 것
## 9:1 나눈다고 했을 때 10%를 안쓰고 있고, 마지막 1~2 epoch에서 10%로 학습하고 90%를 검증으로 사용 => 여전히 성능이 validation과 train이 유지 또는 오른다면 일반화되고 있다는 것
## + 개선 여지가 있다는 것
## 학습해야할 데이터가 남아 있을 때 제출하는건 횟수 낭비다!
## 현실에서는 val과 test를 나누진 않음

## utils는 torch와 겹칠 수 있으므로 우리 폴더명 utils은 util로 작성하기 
## dataloader.py => data.py로 만들고 그 안에 customdataset과 def custom_data_loader로 작성해서 train.ipynb에서는 custom_data_loader만 불러올 수 있도록

train_dataset = CustomDataset("", root_dir='./data', transform=transform)
val_dataset = CustomDataset("", root_dir='./data', transform=transform)
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                                    batch_size = BATCH_SIZE,
                                                    shuffle = False, # 순서가 암기되는것을 막기위해.
                                                    )
val_loader = torch.utils.data.DataLoader(dataset = val_dataset,
                                                    batch_size = BATCH_SIZE,
                                                    shuffle = False, # 테스트 순서 유지.
                                                    )

In [6]:
len(train_dataset)

204

In [7]:
train_dataset[0][0].shape

torch.Size([3, 224, 224])

# Model Def

In [9]:
class AlexNet(nn.Module):
    def __init__(self, num_class=10):
        super(AlexNet, self).__init__()

        self.conv_layer1 = nn.Sequential(
            nn.Conv2d(1, 96, kernel_size=4),
            nn.ReLU(inplace=True),
            nn.Conv2d(96, 96, kernel_size=3),
            nn.ReLU(inplace=True)
        )
        self.conv_layer2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.conv_layer3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

        self.fc_layer1 = nn.Sequential(
            nn.Dropout(),
            nn.Linear(6400, 800),
            nn.ReLU(inplace=True),
            nn.Linear(800, 10)
        )

    def forward(self, x):
        output = self.conv_layer1(x)
        output = self.conv_layer2(output)
        output = self.conv_layer3(output)
        output = torch.flatten(output, 1)
        output = self.fc_layer1(output)
        return output


# Model Load & Loss Function

# TRAIN CODE

In [10]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [11]:
def binary_acc(y_pred, y_test): #이진 분류?

    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    #checkpoint_filepath = 'checkpoint.pth'  # 체크포인트 저장 파일 경로
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [15]:
config = {'epoches': EPOCHS, 'batch_size': BATCH_SIZE, 'learning_rate': lr}
wandb.init(project='my-test-project', config=config)

## 시각화 관련해서
## 다 찍는게 좋음 
## 전체 Loss & Acc
## class별 loss & Acc
## Learning rate
## 데이터가 적은 경우 => 틀린 그림의 idx 뽑아보기
## 최근에 acc보단 F1 score를 더 많이 사용하기도 해서 같이 찍어보면 좋음
## acc와 F1 score 중 둘 중 하나 더 높은 모델들 끼리 비교했을 때 무엇이 더 일반화 성능이 더 좋을지 모름

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kihoon090. Use `wandb login --relogin` to force relogin


In [16]:
from util.checkpoints import save_checkpoint

#checkpoint_interval = 5 ## 5에폭마다 체크포인트 저장
best_val_loss = float('inf')  # 초기값 무한대

# checkpoints 폴더가 없으면 생성
checkpoint_dir = 'checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

for epoch in range(EPOCHS):
    train_loss = 0
    train_acc = 0
    val_loss = 0
    val_acc = 0

    ## Training
    model.train() ## 학습용
    print(f"Training on {epoch}")
    model.train()
    train_correct = 0
    for images, labels in tqdm(train_loader, desc="train"):
        images, labels = images.to(device), labels.to(device)
            # size += len(labels)
        optimizer.zero_grad()
        pred = model(images)
        
        # pred => [0.2, 0.3, 0.5, 0] 
        # label => [0] or [1] or [2] or [3] 
        
        loss = criterion(pred, labels)
        loss.backward()
        optimizer.step()

        #손실 계산
        train_loss += loss.item()
        #train_acc += acc.item()
        
        #정확도 계산
        # pred => argmax()
        # label  => [0] or [1] 
        pred = torch.argmax(pred, dim=1) #이건 다중 분류에서 쓰는 건데
        #acc = binary_acc(pred, labels) #이거는 이진 분륜디
        acc = (pred == labels).sum().item()
        train_correct += acc


    
    #배치별로 평균
    #train_loss = train_loss / len(train_dataset) ## data의 개수로 나눠주는게 아님 => 위에 size를 나눠주는 방식을 사용함 or dataload의 매개변수로 droplast가 있는데, batch 안채워진거 제거
    #train_acc = train_acc / len(train_dataset)
    avg_train_loss = train_loss / len(train_loader)
    train_acc = train_correct / len(train_loader.dataset)  # 전체 데이터셋에 대한 정확도 계산
    print(f'Epoch {epoch+1:03}: | Train Loss: {avg_train_loss:.5f} | Train Acc: {train_acc:.3f}')

    #Wandb로 train 손실, 정확도 기록
    wandb.log({'Train Accuracy': train_acc, 'Train Loss': avg_train_loss, "Epoch": epoch + 1})
    

    ## Validation
    model.eval()
    val_correct = 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="val"):
            images, labels = images.to(device), labels.to(device)

            pred = model(images)

            loss = criterion(pred, labels)
            
            val_loss += loss.item()
            pred = torch.argmax(pred, dim=1)
            #acc = binary_acc(pred, labels)
            val_correct += (pred == labels).sum().item()

            #val_acc += acc.item()

    #배치별로 평균
    val_loss_avg = val_loss / len(val_loader)
    val_acc = val_correct / len(val_loader.dataset) #ㅈ너체 데이터셋에 대한 정확도
    print(f'Epoch {epoch+1:03}: | Val Loss: {val_loss_avg:.5f} | Val Acc: {val_acc:.3f}')
    
    #wandb로 검증 손실 및 정확도 기록
    wandb.log({'Val Accuracy': val_acc, 'Val Loss': val_loss_avg})
    ## 매 epooch마다 validation을 계산하는 것이 의미가 없다. 따라서 train loss가 낮아졌을 때 validation을 진행하는 것이 바람직함.
    ## 큰 데이터를 가지고 학습을 할 때 특히 학습 시간을 줄 일 때 유용함

    # validation이 개선되었을 때만 체크포인트 저장
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        checkpoint_filepath = os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch + 1}.pth')  # 에포크 번호 포함
        save_checkpoint(model, optimizer, epoch, val_loss, checkpoint_filepath)
        print(f"Checkpoint updated at epoch {epoch + 1} and saved as {checkpoint_filepath}")

# 최종 체크포인트 저장
#save_checkpoint(model, optimizer, epoch, train_loss / len(train_loader), 'final_checkpoint.pth')
final_checkpoint_filepath = os.path.join(checkpoint_dir, 'final_checkpoint.pth')
save_checkpoint(model, optimizer, epoch, avg_train_loss, final_checkpoint_filepath)
print(f"Final checkpoint saved as {final_checkpoint_filepath}")

## excel로 실험 관리
## notion or spread sheet

## 들어갈 내용
## 실험자
## 모델에 대한 구체적인 정보 ex) resnet18 resnet19 
## transfer learning을 했다면 freezing을 어디까지 했는지, pretrain은 feature를 뽑아내는 것까지 밖에 못한다. => 뒷 단의 정보 ex) MLP(1024 -> 1024 -> 500, init 방법)를 어떻게 뭘 적용했는가
## scheduler
## Lr
## 등등
## 실험 결과 작성을 할 때
## 짧은 의견을 달아주는 것이 실험 공유가 편함 (붙이는 정보에 포함)
## ex) MLP 4층 이상 쌓아보이니 안 좋음
## ex) batch norm, 
## 추가적인 상대방을 배려할 때 사용되는 GPU 메모리양 
## gpu를 사용할 때 모델을 한번씩 하는 것보다 같은 동시에 올려서 하는게 더 빠름 

## 최소 몇 epoch를 훈련해야겠더라도 알아내야함
## seed number
## image transformation 바꾼게 있다면 포함

## 마지막에 앙상블 
## hard voting => 모든 모델이 동일한 가중치
## soft voting => 모델이 각각 다른 가중치 => 지표(train loss, val loss, acc)가 필요함

## 추가적인 생각
## 학습을 하는데 틀리는 데이터는 분명히 존재할 것이다. 
## validation을 random하게 짜는 것이 좋은게 아닐 수 있다
##   => 사람이 봐도 어려운 문제가 있다고 하자, 이런 것을 맞추는 모델이 똑똑한 것인지 생각해볼 것
##   => 팀내의 정책을 정해서 쉬운 데이터, 어려운 데이터, 매우 어려운 데이터로 분류해서 validation을 구성하는 것이 좋음
##   => 이렇게 해야 데이터의 난이도에 상관 없이 잘 분류하는 모델을 얻을 수 있다.
##   => 완전히 데이터가 많을 때에는 random하게 나눠도 문제 없다.

## ipynb는 간단한 아이디어를 검증하기 위해서 사용할 수 있음 
## 실제 수행은 py 파일에서 하는 것이 맞음 => shell script 자동화 해놓을 수 있음

## 기회가 남는다면 하이퍼파라미터 조정도 시도해볼 수 있을 것이다.

## residual, normalize, CNN N층 => N층 자동 생성

## activaiton.py은 어차피 활성화함수를 구현할게 아니기 때문에 필요 없음

## paperswithcode에서 모델 찾아보고 적용해볼 것

# 자료형 명시
# def __len__(self) -> int: ## 이런걸 전부 포함하도록 하는게 좋음
#     # 데이터 개수 반환 함수
#     return len(self.image_paths)

## baseline code를 어떻게 작성했는지 많이 찾아보고 분석해볼 것

## 목표
## 호철이형 : 전체적인 시도 => 다양한 시도
## 나 : 모델링 + baseline 개선 => 모델링 분석 + baseline code 분석 및 개선
## 재환이형 : 모델 관련해서 어떻게 구현되어 있는지 분석해서 공유
## 유향이 : 큰 모델의 transfer learning, 모델 밑바닥 빌딩 => 데이터 feature 추출과 layer size를 제대로 파악 <= 감각적으로 이해하는 것도 중요하고 다른 팀원들에게 정보를 공유하면 좋을 듯
## 종민이형 : 전체적인 흐름 이해 + data & data augmentation technique 실험 => 시각화부터 시작해서 데이터를 살펴보는 것부터 시작. augmentation이 많아질 수록 직접 시각화하여 어떤 결과인지 확인해보기
##           data augmentation 적용했을 때 무엇이 잘 되었고 잘 되지 못했는지 파악하기, library에 없는 augmentation을 직접 구현해볼 수 도 있을 것
##           mixup을 시도할 때 labeling을 나누는 것 비율
## 소윤이 : 개발의 흐름 이해 + 서버 이용 방법 + 다른 사람 코드 분석해서 추가적인 insight 도출 <= 이건 팀원 모두가 해야함, 즉 모두가 답이 나와야함 <= 포폴에 넣었을 때 면접가서 질문이 들어오면 다 방어할 수 있음
## 없는 내용 : optimization(adam, Radam 등등)/scheduler(cosine scheduler) 부분을 소윤이가 담당해보는 것이 좋아보임 => 잘 모르는 경우가 많음

## 프젝 기간 동안 성장을 해야한다. 따라서 계획이 그만큼 중요한 것. 
## 이론 부분은 언제든지 되돌아가서 볼 수 있다.
## 개개인의 성장을 느끼는 것이 중요.
## 개개인의 시너지를 잘 이용했으면 좋겠다. 

## data augmentation에 대해서
## data transform이라고 부르는 것이 더 맞음
## 증강된 데이터로만 학습해도 문제가 없는 것인가? 
# => offline(새로운 이미지를 생성) augmentation(transform을 적용했을 때 성능이 오르는게 확정적일 때 사용)과 online(확률적 적용, 불러올 때 적용) augmentation으로 나눈다.
 
## RGB와 Grayscale
## RGB가 superset이므로 일반적으로 Grayscale보다 RGB로 하는게 맞지만, 이는 확인해보는게 좋음

## 프로젝트 역할 구분
## 크게 구분하면 => 위에 나왔던 내용

## outlier가 치명적이면 버리고 필요하면 사용하는게 맞다.

 

Training on 0


train: 100%|██████████| 7/7 [00:03<00:00,  2.28it/s]


Epoch 000: | Loss: 0.04759 | Acc: 0.863


val: 100%|██████████| 7/7 [00:02<00:00,  2.62it/s]


Epoch 000: | Loss: 0.04763 | Acc: 0.784
Checkpoint saved at checkpoints\checkpoint_epoch_1.pth
Checkpoint updated at epoch 1 and saved as checkpoints\checkpoint_epoch_1.pth
Training on 1


...

Epoch 032: | Loss: 0.04751 | Acc: 0.784
Training on 33


train: 100%|██████████| 7/7 [00:02<00:00,  2.38it/s]


Epoch 033: | Loss: 0.04758 | Acc: 0.838


val:  29%|██▊       | 2/7 [00:01<00:03,  1.49it/s]


KeyboardInterrupt: 